In [64]:
import os
import numpy as np 
import pandas as pd 
from sklearn.metrics import log_loss

In [71]:
feat_names = []
#feat_names.append("orig+TLvgg16+Adam")
#feat_names.append("orig+TLres50")
#feat_names.append("orig1+TLvgg16+Adam")
feat_names.append("orig1+TLres50")

#feat_names.append("all+TLvgg16+Adam")

#feat_names.append("trans+TLvgg16+Adam")
#feat_names.append("trans+TLres50")
feat_names.append("trans1+TLvgg16+Adam")
#feat_names.append("trans1+TLres50")

#feat_names.append("norm+TLvgg16")
#feat_names.append("norm+TLres50")
#feat_names.append("norm1+TLvgg16")
#feat_names.append("norm1+TLres50")

feat_names.append("normf1+TLvgg16")
#feat_names.append("norm+lenet")

#feat_names.append("ctt+xgb")
#feat_names.append("picf+xgb")
#feat_names.append("pca+lgb")

feat_names.append("mix+lr")
#feat_names.append("mix+xgb")
feat_names.append("mix+lgb")
feat_names.append("mix+svm")

In [72]:
train_org = pd.read_json('../../input/train.json')
y = train_org['is_iceberg'].values

In [83]:
valids = [pd.read_csv('../../subm/'+f+'_submit.csv') for f in feat_names]
valid = pd.merge(valids[0],valids[1] , how='left',on='id')
for i in range(2,len(feat_names)):
    valid = pd.merge(valid,valids[i],how='left',on='id')
cols = ['id']+feat_names
valid.columns = cols
#valid['is_iceberg'] = y

In [84]:
valid.corr()

,orig1+TLres50,trans1+TLvgg16+Adam,normf1+TLvgg16,mix+lr,mix+lgb,mix+svm
orig1+TLres50,1.000000,0.911887,0.888034,0.528433,0.792082,0.624787
trans1+TLvgg16+Adam,0.911887,1.000000,0.936294,0.367383,0.749749,0.495229
normf1+TLvgg16,0.888034,0.936294,1.000000,0.394507,0.722274,0.483206
mix+lr,0.528433,0.367383,0.394507,1.000000,0.565103,0.747470
mix+lgb,0.792082,0.749749,0.722274,0.565103,1.000000,0.652915
mix+svm,0.624787,0.495229,0.483206,0.747470,0.652915,1.000000


In [90]:
lrpre = pd.read_csv('../../subm/ensemble/lr9_submit.csv')
valid['lr9'] = lrpre['is_iceberg']

In [91]:
xgbpre = pd.read_csv('../../subm/ensemble/xgb9_submit.csv')
valid['xgb'] = xgbpre['is_iceberg']

In [92]:
svmpre = pd.read_csv('../../subm/ensemble/svm9_submit.csv')
valid['svm'] = svmpre['is_iceberg']

In [93]:
knnpre = pd.read_csv('../../subm/ensemble/knn9_submit.csv')
valid['knn'] = knnpre['is_iceberg']

In [94]:
valid['avg2'] = pd.read_csv('../../subm/ensemble/avarage_submit.csv')['is_iceberg']
valid['mini2'] = pd.read_csv('../../subm/ensemble/minimize_submit.csv')['is_iceberg']

In [62]:
def get_clip(df,y,ep):
    for i in ['lr9','xgb','svm','knn','avg2','mini2']:
        print(i,'befor clip: ',log_loss(y,df[i]))
        print(i,'after clip: ',log_loss(y,df[i].clip(ep,1-ep)))

In [67]:
get_clip(valid,y,1e-2)

lr9 befor clip:  0.125300203511
lr9 after clip:  0.125619215431
xgb befor clip:  0.115518444416
xgb after clip:  0.11777876035
svm befor clip:  0.127908606094
svm after clip:  0.127572540465
knn befor clip:  0.124161028028
knn after clip:  0.124303570469
avg2 befor clip:  0.117952243672
avg2 after clip:  0.119594208724
mini2 befor clip:  0.114476482079
mini2 after clip:  0.116529323072


In [50]:
iceb = valid[valid.is_iceberg==1]
ship = valid[valid.is_iceberg==0]

In [20]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
valid

,id,orig1+TLres50,trans1+TLvgg16+Adam,normf1+TLvgg16,mix+lr,mix+xgb,mix+lgb,mix+svm,is_iceberg,lr9
0,dfd5f913,0.001405,0.007866,0.028444,0.000733,0.003447,0.001917,0.009039,0,0.006675
1,e25388fd,0.000000,0.000000,0.000206,0.000000,0.000513,0.000046,0.000006,0,0.006569
2,58b2aaa0,0.859876,0.858316,0.833085,0.962498,0.989944,0.996452,0.983096,1,0.977428
3,4cfc3a18,0.000337,0.003162,0.002622,0.000008,0.009831,0.001263,0.000083,0,0.006672
4,271f93f4,0.006346,0.000255,0.017979,0.121713,0.073113,0.046099,0.096352,0,0.006648
5,b51d18b5,0.999778,0.996920,0.998533,0.997964,0.999597,0.999820,0.999993,1,0.990575
6,31da1a04,0.938238,0.967257,0.867363,0.907990,0.985218,0.991858,0.982883,1,0.984629
7,56929c16,0.054986,0.479746,0.249921,0.216860,0.168032,0.169076,0.658739,0,0.106837
8,525ab75c,0.006049,0.001392,0.024439,0.000000,0.002053,0.000948,0.000055,0,0.004784
9,192f56eb,0.287602,0.081951,0.155503,0.714398,0.188653,0.462956,0.786655,0,0.142886


In [23]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:8].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:8].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:8].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:8].median(axis=1)

In [24]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.8
cutoff_hi = 0.2

In [25]:
from sklearn.metrics import log_loss

In [26]:
log_loss(y,concat_sub['is_iceberg_mean'])

0.14532914707895817

In [27]:
log_loss(y,concat_sub['is_iceberg_median'])

0.13677715088745776

In [30]:
a = np.where(np.all(concat_sub.iloc[:,1:8] > cutoff_lo, axis=1), 1, 
                                    np.where(np.all(concat_sub.iloc[:,1:8] < cutoff_hi, axis=1),
                                             0, concat_sub['is_iceberg_median']))
log_loss(y,a)

0.22602436341779417

In [31]:
a=np.where(np.all(concat_sub.iloc[:,1:8] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:8] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_mean']))
log_loss(y,a)

0.14369445106336748

In [32]:
a=np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))
log_loss(y,a)

0.13851888085083117

In [54]:
a=np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             (valid['lr9']+valid['xgb'])/2))
log_loss(y,a)

0.123000144415538

In [34]:
a=np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             valid['trans1+TLvgg16+Adam']))
log_loss(y,a)

0.14840385060244043

In [35]:
a=np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    .5*concat_sub['is_iceberg_max']+.5, 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             .5*concat_sub['is_iceberg_min']+.5, 
                                             valid['lr9']))
log_loss(y,a)

0.37899478771471873

In [1]:
X = valid.loc[:,['orig1+TLres50', 'trans1+TLvgg16+Adam','normf1+TLvgg16','mix+lr','mix+lgb','mix+svm']]


NameError: name 'valid' is not defined

In [46]:
Xnp.sum((X.iloc[:,:]>0.5).astype(int),axis=1)/6.0

0       0.000000
1       0.000000
2       1.000000
3       0.000000
4       0.000000
5       1.000000
6       1.000000
7       0.166667
8       0.000000
9       0.333333
10      1.000000
11      0.000000
12      1.000000
13      1.000000
14      0.000000
15      0.000000
16      0.500000
17      0.000000
18      0.000000
19      0.333333
20      0.000000
21      0.666667
22      0.000000
23      1.000000
24      0.000000
25      1.000000
26      0.333333
27      0.000000
28      0.833333
29      0.000000
30      0.000000
31      0.000000
32      1.000000
33      0.833333
34      1.000000
35      1.000000
36      1.000000
37      0.000000
38      1.000000
39      1.000000
40      1.000000
41      0.000000
42      0.000000
43      0.000000
44      0.000000
45      0.000000
46      1.000000
47      1.000000
48      0.666667
49      1.000000
50      1.000000
51      1.000000
52      0.000000
53      1.000000
54      1.000000
55      0.166667
56      0.000000
57      0.166667
58      1.0000

In [63]:
np.sum(np.all(X.iloc[:,:] > .9, axis=1).astype(int))

342

In [64]:
np.sum(np.all(X.iloc[:,:] < .1, axis=1).astype(int))

526

In [68]:
np.sum(np.all(X.iloc[:,0:3] > .8, axis=1).astype(int))

552

In [62]:
np.sum(np.all(X.iloc[:,0:3] < .2, axis=1).astype(int))

672

In [69]:
iceb.shape

(753, 11)

In [70]:
ship.shape

(851, 11)

In [73]:
a = np.all(X.iloc[:,:] > .8, axis=1).astype(int) + np.all(X.iloc[:,:] < .1, axis=1).astype(int)
valid['type'] = a

In [82]:
wanted = valid[valid.type==0]
wanted.drop(['id','lr9','xgb'],axis=1,inplace=True)
iceb_wanted = wanted[wanted.is_iceberg==1]
ship_wanted = wanted[wanted.is_iceberg==0]

/Users/afu/anaconda2/envs/ml/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [83]:
wanted.shape

(625, 9)

In [84]:
iceb_wanted.shape

(299, 9)

In [85]:
ship_wanted.shape

(326, 9)

In [86]:
import matplotlib.pyplot as plt

In [95]:
a=2

In [181]:
def gen_type(df):
    hig = .8
    mid = .5
    low = .2
    if df[0]>hig and df[1]>hig and df[2]>hig and df[3]>hig and df[4]>hig and df[5]>hig:
        return 10
    if df[0]<low and df[1]<low and df[2]<low and df[3]<low and df[4]<low and df[5]<low:
        return 11
    
    if df[0]>hig and df[1]>hig and df[2]>hig and df[3]<low and df[4]<low and df[5]<low:
        return 20
    if df[0]<low and df[1]<low and df[2]<low and df[3]>hig and df[4]>hig and df[5]>hig:
        return 21
    
    if df[0]>hig and df[1]>hig and df[2]>hig and df[3]>mid and df[4]>mid and df[5]>mid:
        return 30
    if df[0]<low and df[1]<low and df[2]<low and df[3]<mid and df[4]<mid and df[5]<mid:
        return 31
    
    if df[0]>hig and df[1]>hig and df[2]>hig and df[3]>low and df[4]>low and df[5]>low:
        return 40
    if df[0]<low and df[1]<low and df[2]<low and df[3]<hig and df[4]<hig and df[5]<hig:
        return 41
    
    if df[0]>mid and df[1]>mid and df[2]>mid and df[3]>mid and df[4]>mid and df[5]>mid:
        return 50
    if df[0]<mid and df[1]<mid and df[2]<mid and df[3]<mid and df[4]<mid and df[5]<mid:
        return 51
    
    if df[0]>mid and df[1]>mid and df[2]>mid and df[3]<low and df[4]<low and df[5]<low:
        return 60
    if df[0]<mid and df[1]<mid and df[2]<mid and df[3]>hig and df[4]>hig and df[5]>hig:
        return 61 

    if np.sum((df[:6]>mid).astype(int))==5:
        return 70
    if np.sum((df[:6]<mid).astype(int))==5:
        return 71
    
    if np.sum((df[:6]>mid).astype(int))==4 and np.sum((df[:6]<low).astype(int))==0:
        return 80
    if np.sum((df[:6]<mid).astype(int))==5 and np.sum((df[:6]>hig).astype(int))==0:
        return 81
    return 0

In [176]:
my.apply(gen_type,1)

(8,)
1
0
(8,)
2
0
(8,)
3
0
(8,)
2
0
(8,)
4
0
(8,)
2
0
(8,)
5
0
(8,)
5
0
(8,)
4
0
(8,)
1
0
(8,)
1
0
(8,)
3
0
(8,)
1
0
(8,)
2
0
(8,)
2
0
(8,)
2
0
(8,)
3
0
(8,)
3
0
(8,)
3
0
(8,)
5
0
(8,)
5
0
(8,)
2
0
(8,)
1
0
(8,)
1
0
(8,)
4
0
(8,)
1
0
(8,)
2
0
(8,)
2
0
(8,)
4
0
(8,)
5
0
(8,)
2
0
(8,)
3
0
(8,)
3
0
(8,)
2
0
(8,)
5
0
(8,)
2
0
(8,)
1
0
(8,)
1
0
(8,)
1
0
(8,)
2
0
(8,)
5
0
(8,)
3
0
(8,)
5
0
(8,)
5
0
(8,)
3
0
(8,)
3
0
(8,)
5
0
(8,)
1
0
(8,)
1
0
(8,)
5
0
(8,)
5
0
(8,)
2
0
(8,)
1
0
(8,)
0
0
(8,)
2
0
(8,)
1
0
(8,)
5
0
(8,)
2
0
(8,)
1
0
(8,)
2
0
(8,)
2
0
(8,)
5
0
(8,)
5
0
(8,)
1
0
(8,)
5
0
(8,)
4
0
(8,)
5
0
(8,)
4
0
(8,)
1
0
(8,)
2
0
(8,)
4
0
(8,)
2
0
(8,)
2
0
(8,)
3
0
(8,)
2
0
(8,)
3
0
(8,)
2
0
(8,)
3
0
(8,)
5
0
(8,)
1
0
(8,)
1
0
(8,)
4
0
(8,)
1
0
(8,)
2
0
(8,)
1
0
(8,)
4
0
(8,)
3
0
(8,)
1
0
(8,)
3
0
(8,)
5
0
(8,)
4
0
(8,)
5
0
(8,)
4
0
(8,)
2
0
(8,)
4
0
(8,)
5
0
(8,)
1
0
(8,)
5
0
(8,)
5
0
(8,)
1
0
(8,)
2
0
(8,)
3
0
(8,)
2
0
(8,)
4
0
(8,)
3
0
(8,)
5
0
(8,)
4
0
(8,)
1
0
(8,)
5
0
(8,)
1
0
(8,)
5
0
(

0       11
1       11
2       10
3       11
4       11
5       10
6       10
7       71
8       11
9        0
10      50
11      11
12      50
13      50
14      11
15      11
16       0
17      31
18      51
19       0
20      11
21       0
22      11
23      10
24      11
25      10
26       0
27      11
28      70
29      11
30      11
31      11
32      30
33      70
34      10
35      10
36      10
37      11
38      50
39      10
40      10
41      11
42      11
43      11
44      11
45      11
46      10
47      10
48       0
49      10
50      10
51      10
52      51
53      30
54      50
55      71
56      11
57      71
58      30
59      11
60       0
61      50
62      11
63      51
64      11
65      71
66      30
67       0
68      10
69      10
70      10
71      41
72      51
73      10
74      10
75      10
76       0
77      10
78      10
79      11
80      11
81      30
82      50
83      30
84      10
85      10
86      10
87      11
88      50
89      11
90      30

In [121]:
my = valid.loc[:,['orig1+TLres50','trans1+TLvgg16+Adam','normf1+TLvgg16','mix+lr','mix+lgb','mix+svm','is_iceberg']].astype(float)
my.apply(gen_type,1)

0       (0, 0)
1       (0, 0)
2       (1, 2)
3       (0, 0)
4       (0, 0)
5       (1, 2)
6       (1, 2)
7       (0, 0)
8       (0, 0)
9       (0, 0)
10      (0, 0)
11      (0, 0)
12      (0, 0)
13      (0, 0)
14      (0, 0)
15      (0, 0)
16      (0, 0)
17      (0, 0)
18      (0, 0)
19      (0, 0)
20      (0, 0)
21      (0, 0)
22      (0, 0)
23      (1, 2)
24      (0, 0)
25      (1, 2)
26      (0, 0)
27      (0, 0)
28      (0, 0)
29      (0, 0)
30      (0, 0)
31      (0, 0)
32      (0, 0)
33      (0, 0)
34      (1, 2)
35      (1, 2)
36      (1, 2)
37      (0, 0)
38      (0, 0)
39      (1, 2)
40      (1, 2)
41      (0, 0)
42      (0, 0)
43      (0, 0)
44      (0, 0)
45      (0, 0)
46      (1, 2)
47      (1, 2)
48      (0, 0)
49      (1, 2)
50      (1, 2)
51      (1, 2)
52      (0, 0)
53      (0, 0)
54      (0, 0)
55      (0, 0)
56      (0, 0)
57      (0, 0)
58      (0, 0)
59      (0, 0)
60      (0, 0)
61      (0, 0)
62      (0, 0)
63      (0, 0)
64      (0, 0)
65      (0, 0)
66      (0

In [182]:
my['type'] = my.apply(gen_type,1)

In [183]:
my[my.type==0]

,orig1+TLres50,trans1+TLvgg16+Adam,normf1+TLvgg16,mix+lr,mix+lgb,mix+svm,is_iceberg,type
9,0.287602,0.081951,0.155503,0.714398,0.462956,0.786655,0.0,0
16,0.395006,0.134649,0.123334,0.688618,0.830886,0.826910,0.0,0
19,0.471400,0.758480,0.499664,0.450051,0.620073,0.193130,1.0,0
21,0.084159,0.601634,0.789117,0.143754,0.871721,0.664435,1.0,0
26,0.163231,0.131698,0.054178,0.980769,0.402647,0.777715,1.0,0
60,0.338512,0.590481,0.454658,0.679037,0.689164,0.423047,1.0,0
67,0.257403,0.064057,0.072478,0.489256,0.552255,0.695227,0.0,0
76,0.294312,0.599969,0.814481,0.037950,0.288296,0.192871,0.0,0
92,0.672140,0.526667,0.112964,0.130852,0.059810,0.108984,0.0,0
95,0.616421,0.592385,0.794065,0.069655,0.032795,0.339758,0.0,0


In [184]:
my[my.type==0].shape

(100, 8)

In [185]:
my['lr_pred'] = valid['lr9']

In [186]:
my.to_csv('mytype.csv', float_format='%.6f', index=False)

In [12]:
id = valid.loc[:,'id']

In [13]:
valid.drop(['id'],axis=1,inplace=True)

In [14]:
valid['id']=id

In [15]:
valid.head()

,orig1+TLres50,trans1+TLvgg16+Adam,normf1+TLvgg16,mix+lr,mix+lgb,mix+svm,is_iceberg,lr9,id
0,0.001405,0.007866,0.028444,0.000733,0.001917,0.009039,0,0.006688,dfd5f913
1,0.000000,0.000000,0.000206,0.000000,0.000046,0.000006,0,0.006576,e25388fd
2,0.859876,0.858316,0.833085,0.962498,0.996452,0.983096,1,0.977436,58b2aaa0
3,0.000337,0.003162,0.002622,0.000008,0.001263,0.000083,0,0.006679,4cfc3a18
4,0.006346,0.000255,0.017979,0.121713,0.046099,0.096352,0,0.006637,271f93f4


In [16]:
valid.to_csv('myvalid.csv', float_format='%.6f', index=False)

In [17]:
valids = [pd.read_csv('../../subm/'+f+'_submit.csv') for f in feat_names]
valid = pd.merge(valids[0],valids[1] , how='left',on='id')
for i in range(2,len(feat_names)):
    valid = pd.merge(valid,valids[i],how='left',on='id')
cols = ['id']+feat_names
valid.columns = cols


In [18]:
lrpre = pd.read_csv('../../subm/ensemble/lr9_submit.csv')
valid['lr9'] = lrpre['is_iceberg']

In [95]:
id = valid.loc[:,'id']
valid.drop(['id'],axis=1,inplace=True)
valid['id']=id
valid.head()

,orig1+TLres50,trans1+TLvgg16+Adam,normf1+TLvgg16,mix+lr,mix+lgb,mix+svm,lr9,xgb,svm,knn,avg2,mini2,id
0,0.042638,0.012704,0.047947,0.004939,0.084877,0.109784,0.010339,0.041586,0.009558,0.048438,0.027480,0.040736,5941774d
1,0.635144,0.479274,0.385688,0.999584,0.883075,0.989249,0.807971,0.903653,0.841273,0.867362,0.855065,0.893923,4023181e
2,0.011445,0.000052,0.001072,1.000000,0.011936,0.273804,0.020771,0.017792,0.010907,0.057899,0.026842,0.024811,b20200e4
3,0.999374,0.996001,0.998098,0.963870,0.995109,0.984362,0.989768,0.994674,0.982609,0.999000,0.991513,0.996062,e7f018bb
4,0.509366,0.042978,0.022157,0.999994,0.504518,0.885033,0.187754,0.433252,0.190194,0.224531,0.258933,0.378449,4371c8c3


In [96]:
valid.to_csv('mytest.csv', float_format='%.6f', index=False)

In [97]:
valid.shape

(8424, 13)

In [26]:
a = pd.read_csv('mytest.csv')
a.head()

,orig1+TLres50,trans1+TLvgg16+Adam,normf1+TLvgg16,mix+lr,mix+lgb,mix+svm,lr9,id
0,0.042638,0.012704,0.047947,0.004939,0.084877,0.123180,0.010431,5941774d
1,0.635144,0.479274,0.385688,0.999584,0.883075,0.994627,0.807077,4023181e
2,0.011445,0.000052,0.001072,1.000000,0.011936,0.515743,0.028135,b20200e4
3,0.999374,0.996001,0.998098,0.963870,0.995109,0.977835,0.989663,e7f018bb
4,0.509366,0.042978,0.022157,0.999994,0.504518,0.948791,0.199618,4371c8c3


In [27]:
isi = a.loc[:,'is_iceberg']
a.drop(['is_iceberg'],axis=1,inplace=True)
a['is_iceberg'] = isi
a.head()

KeyError: 'the label [is_iceberg] is not in the [columns]'

In [25]:
a.to_csv('mytest.csv', float_format='%.6f', index=False)

In [69]:
valid[(valid.avg2>.8)&(valid.is_iceberg<.5)]

,id,orig1+TLres50,all+TLvgg16+Adam,trans1+TLvgg16+Adam,normf1+TLvgg16,mix+lr,mix+lgb,mix+svm,is_iceberg,lr9,xgb,svm,knn,avg2,mini2
161,2f881d78,0.992831,0.931475,0.964832,0.971465,0.998016,0.984880,0.991665,0,0.988656,0.993125,0.983176,0.999000,0.990989,0.994943
333,74c92938,0.937305,0.917838,0.925763,0.921871,0.770765,0.678706,0.785012,0,0.954266,0.778645,0.971729,0.891415,0.899014,0.813897
350,1303a495,0.978347,0.989927,0.934119,0.903692,0.734713,0.917766,0.545961,0,0.962009,0.922343,0.971122,0.950435,0.951477,0.932200
364,5a501d33,0.928160,0.944489,0.829317,0.961569,0.693024,0.932323,0.757553,0,0.966156,0.955035,0.978943,0.890095,0.947557,0.945930
562,55a724b1,0.835171,0.627032,0.844869,0.638103,0.502649,0.408982,0.886780,0,0.832006,0.948207,0.917300,0.869438,0.891738,0.932781
651,f613255d,0.375417,0.480621,0.437338,0.722559,0.522890,0.938852,0.870892,0,0.832766,0.830069,0.853440,0.865176,0.845363,0.839369
661,786ee92d,0.901814,0.496225,0.702368,0.348486,0.756717,0.789169,0.749184,0,0.848299,0.716030,0.884899,0.906328,0.838889,0.763970
684,fd1f0c9b,0.838909,0.982526,0.896258,0.949342,0.719365,0.831304,0.784393,0,0.966841,0.651807,0.978792,0.871882,0.867330,0.715979
791,93beaaaa,0.961608,0.888318,0.973652,0.781880,0.914637,0.985917,0.802822,0,0.975610,0.990043,0.980408,0.999000,0.986265,0.992452
795,e8200704,0.793969,0.979467,0.978606,0.969772,0.713314,0.902842,0.931186,0,0.976712,0.971148,0.980246,0.967878,0.973996,0.972532


In [98]:
valid[(valid.avg2<.2)&(valid.is_iceberg>.5)]

AttributeError: 'DataFrame' object has no attribute 'is_iceberg'

In [99]:
valid.corr()

,orig1+TLres50,trans1+TLvgg16+Adam,normf1+TLvgg16,mix+lr,mix+lgb,mix+svm,lr9,xgb,svm,knn,avg2,mini2
orig1+TLres50,1.000000,0.911887,0.888034,0.528433,0.792082,0.624787,0.918553,0.886913,0.913552,0.917527,0.915789,0.900322
trans1+TLvgg16+Adam,0.911887,1.000000,0.936294,0.367383,0.749749,0.495229,0.926714,0.862660,0.920285,0.915353,0.912806,0.882109
normf1+TLvgg16,0.888034,0.936294,1.000000,0.394507,0.722274,0.483206,0.925829,0.866012,0.916081,0.924141,0.914577,0.885971
mix+lr,0.528433,0.367383,0.394507,1.000000,0.565103,0.747470,0.511290,0.575778,0.489808,0.541126,0.533532,0.567334
mix+lgb,0.792082,0.749749,0.722274,0.565103,1.000000,0.652915,0.880292,0.900888,0.866632,0.855311,0.882341,0.896140
mix+svm,0.624787,0.495229,0.483206,0.747470,0.652915,1.000000,0.634219,0.688484,0.634679,0.646388,0.655906,0.681621
lr9,0.918553,0.926714,0.925829,0.511290,0.880292,0.634219,1.000000,0.970246,0.996963,0.993448,0.997433,0.982847
xgb,0.886913,0.862660,0.866012,0.575778,0.900888,0.688484,0.970246,1.000000,0.964306,0.965747,0.982385,0.997908
svm,0.913552,0.920285,0.916081,0.489808,0.866632,0.634679,0.996963,0.964306,1.000000,0.992529,0.995706,0.978411
knn,0.917527,0.915353,0.924141,0.541126,0.855311,0.646388,0.993448,0.965747,0.992529,1.000000,0.995177,0.980365
